# LlamaIndex Bottoms-Up Development - Embeddings
Embeddings are numerical representations of text. To generate embeddings for text, a specific model is required.

In LlamaIndex, the default embedding model is `text-embedding-ada-002` from OpenAI. You can also leverage any embedding models offered by Langchain and Huggingface using our `LangchainEmbedding` wrapper.

In this notebook, we cover the low-level usage for both OpenAI embeddings and HuggingFace embeddings.

In [1]:
from dotenv import load_dotenv
load_dotenv()       

True

In [2]:
from llama_index.embeddings.openai import OpenAIEmbedding
openai_embedding = OpenAIEmbedding()
embed = openai_embedding.get_text_embedding("hello world!")
print(len(embed))
print(embed[:10])

1536
[-0.0077317021787166595, -0.0055575682781636715, -0.016167471185326576, -0.03340408205986023, -0.016780272126197815, -0.003158524166792631, -0.015606825239956379, -0.0019084787927567959, -0.003049328690394759, -0.026989247649908066]


## Custom Embeddings
While we can integrate with any embeddings offered by Langchain, you can also implement the `BaseEmbedding` class and run your own custom embedding model!

For this, we will use the `InstructorEmbedding` pip package, in order to run `hkunlp/instructor-large` model found here: https://huggingface.co/hkunlp/instructor-large

In [3]:
# Instal dependencies
# !pip install InstructorEmbedding torch transformers sentence_transformers

In [4]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'How to bake a chocolate cake'),
    get_detailed_instruct(task, 'Symptoms of the flu')
]
# No need to add instruction for retrieval documents
passages = [
    "To bake a delicious chocolate cake, you'll need the following ingredients: all-purpose flour, sugar, cocoa powder, baking powder, baking soda, salt, eggs, milk, vegetable oil, and vanilla extract. Start by preheating your oven to 350°F (175°C). In a mixing bowl, combine the dry ingredients (flour, sugar, cocoa powder, baking powder, baking soda, and salt). In a separate bowl, whisk together the wet ingredients (eggs, milk, vegetable oil, and vanilla extract). Gradually add the wet mixture to the dry ingredients, stirring until well combined. Pour the batter into a greased cake pan and bake for 30-35 minutes. Let it cool before frosting with your favorite chocolate frosting. Enjoy your homemade chocolate cake!",
    "The flu, or influenza, is an illness caused by influenza viruses. Common symptoms of the flu include a high fever, chills, cough, sore throat, runny or stuffy nose, body aches, headache, fatigue, and sometimes nausea and vomiting. These symptoms can come on suddenly and are usually more severe than the common cold. It's important to get plenty of rest, stay hydrated, and consult a healthcare professional if you suspect you have the flu. In some cases, antiviral medications can help alleviate symptoms and reduce the duration of the illness."
]

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-2_R')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-2_R')

# get the embeddings
max_length = 4096
input_texts = queries + passages
batch_dict = tokenizer(input_texts, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())
# [[40.132083892822266, 25.032529830932617], [15.006855010986328, 39.93733215332031]]


tokenizer_config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[[79.51094818115234, 32.04499816894531], [30.605417251586914, 74.47244262695312]]


Test the embeddings! Instructor embeddings work by telling it to represent text in a particular domain. 

This makes sense for our llama-docs-bot, since we are search very specific documentation!

Let's quickly test to make sure everything works.

In [5]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

Looks good! But we can see the output is batched (i.e. a list of lists), so we need to undo the batching in our implementation!

There are only 4 methods we need to implement below.

In [19]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR
from llama_index.embeddings.base import BaseEmbedding

class InstructorEmbeddings(BaseEmbedding):
    def __init__(
        self, 
        instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent the Computer Science text for retrieval:",
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name)
        self._instruction = instruction
        super().__init__(**kwargs)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = model.encode([[self._instruction, query]])
        return embeddings[0].tolist()
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = model.encode([[self._instruction, text]])
        return embeddings[0].tolist() 
    
    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = model.encode([[self._instruction, text] for text in texts])
        return embeddings.tolist()

In [20]:
# set the batch size to 1 to avoid memory issues
# if you have a large GPU, you can increase this
instructor_embeddings = InstructorEmbeddings(embed_batch_size=1)

load INSTRUCTOR_Transformer
max_seq_length  512


In [21]:
embed = instructor_embeddings.get_text_embedding("How do I create a vector index?")
print(len(embed))
print(embed[:10])

768
[0.003987083211541176, 0.01212295051664114, 0.0026905445847660303, 0.015817083418369293, -0.0055559673346579075, 0.03673828765749931, 0.010727006942033768, 0.006661377381533384, -0.0392913743853569, 0.013146862387657166]


## Custom Embeddings w/ LlamaIndex

Since Instructor embeddings have a max length of 512, we set the chunk size to 512 as well.

However, if the emebddings are longer, there will not be an error, but only the first 512 tokens will be captured!

In [22]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=instructor_embeddings, chunk_size=512)
set_global_service_context(service_context)

In [9]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

from llama_docs_bot.indexing import create_query_engine

# remove any existing indices
# !rm -rf ./*_index

query_engine = create_query_engine()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
response = query_engine.query('What is the Sub Question query engine?')
response.print_response_stream()

The Sub Question query engine is a system that breaks down complex queries into smaller subquestions through a process called query decomposition. It analyzes the query and identifies different parts or subqueries within it. Each subquery is then routed to a specific subindex within a composed graph, which represents a subset of the overall knowledge corpus. By transforming the original query into simpler subquestions, the engine is able to provide more suitable and targeted answers from the data. This approach is particularly useful for handling complex queries that require knowledge augmentation.

In [11]:
print(response.get_formatted_sources(length=256))

> Source (Node id: 313d0f40-e2b8-467c-841d-311b5b592e34): Sub question: How does the Sub Question query engine work?
Response: The Sub Question query engine works by breaking down a complex query into smaller subquestions. This is done through a process called query decomposition. The engine analyzes the query...

> Source (Node id: e14dc321-291f-4d8a-a0b3-6b6a6e6c0259): Sub question: What are the different components of the Sub Question query engine?
Response: The different components of the Sub Question query engine are:
1. Single-step query decomposition: This component transforms a complicated question into a simple...

> Source (Node id: c27eeb80-53ec-42df-a45c-3e170dcf7b2c): Sub question: How can I configure the Sub Question query engine?
Response: To configure the Sub Question query engine, you can follow these steps:

1. Identify the specific query transformation technique you want to use. In this case, it would be the si...


### Compare to default embeddings

Note that an index must be using the same embedding model at query time that was used to create the index.

So below, we delete the existing indicies and rebuild them using OpenAI embeddings.

In [12]:
from llama_index.embeddings.openai import OpenAIEmbedding

service_context = ServiceContext.from_defaults(llm=llm, embed_model=OpenAIEmbedding(), chunk_size=512)
set_global_service_context(service_context)

# delete old vector index so we can re-create it
!rm -rf ./*_index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
query_engine = create_query_engine()

response = query_engine.query('What is the Sub Question query engine?')
response.print_response_stream()

The Sub Question query engine is a system that breaks down complex questions into smaller subqueries. It uses a query decomposition feature to transform the complicated question into a simpler one over a data collection. By routing the query to multiple subindexes within a composed graph, the engine provides sub-answers to the original question. It consists of components such as a query engine, natural language query input, rich response output, indices, retrievers, composed graph, and query decomposition. The engine can be configured by defining sub-questions, creating a query engine, configuring retrievers, composing the query engine, and testing and refining the system.

In [14]:
print(response.get_formatted_sources(length=256))

> Source (Node id: d3467693-43d5-4a9e-871c-b425d94e2285): Sub question: How does the Sub Question query engine work?
Response: The Sub Question query engine works by breaking down a complex question into smaller subqueries. It uses a single-step query decomposition feature to transform the complicated question...

> Source (Node id: 392bd3f5-83ad-4728-ad66-fe1cca196689): Sub question: What are the different components of the Sub Question query engine?
Response: The different components of the Sub Question query engine are:

1. Query engine: It is a generic interface that allows users to ask questions over their data.

2...

> Source (Node id: e9dca9c4-62b8-492e-8918-75a14eb83e75): Sub question: How can I configure the Sub Question query engine?
Response: To configure the Sub Question query engine, you need to follow these steps:

1. Define the sub-questions: Determine the specific questions you want the query engine to answer. Th...


# Conclusion
In this notebook, we showed how to use the low-level embeddings, as well as how to create your own embeddings class.

If you wanted to use these embeddings in your project (which we will be doing in future guides!), you can use the sample example below.